# Aquaculture and Temperature

**CONTRIBUTORS** (Alphabetically)  
Andrew Anastasiades | andrew-ana  
Caroline Larry | carolinelarry  
Diana Kennen | dlkennen  
Niraj Khatri | MahatmaK  
Tianyue Yang | yangty152  
  
**PROJECT DESCRIPTION**  
This notebook prepares distinct datasets for loading into a relational database.  
  
The original data was retreived from data.world and the files chronicle 
- US Imports and Exports of Aquacultural Products from 1989 to 2016  
- Average temperatures in major cities from 1743 to 2013.


## Dependencies and File Paths

In [1]:
# Dependencies
import pandas as pd #Data Manipulation
import os #Interoperability
from rename_fields import * #Dictionaries to rename fields before loading

# Import File Paths
temp_csv = os.path.join("Data","Land_Temp","land_temps.csv")
export_csv = os.path.join("Data", "Aquaculture", "Exports.csv")
import_csv = os.path.join("Data", "Aquaculture", "Imports.csv")

# Ouput File Paths
TABLE_country = os.path.join("Output", "TABLE_country.csv")
TABLE_fish = os.path.join("Output", "TABLE_fish.csv")
TABLE_commercial = os.path.join("Output", "TABLE_commercial.csv")
TABLE_temp = os.path.join("Output", "TABLE_temp.csv")

## Load and Clean Imports/Exports 
First, Load Data, then begin cleaning  
  
**CLEANING** 
1. Combine imports and exports vertically
2. Drop 'SOURCE_ID' column (It is made redundant by other features)
3. Split 'ATTRIBUTE_DESC'into two more meaningful columns: 'DIRECTION' (Import/Export) and 'MEASURE' (QTY or VLU)
4. Add 'DIRECTION' and 'MEASURE' to the combined commercial_df
5. Drop 'ATTRIBUTE_DESC' as it is no longer needed
6. Fill empty HS_CODE with 404

In [2]:
# LOAD DATA
exports_df = pd.read_csv(export_csv)
imports_df = pd.read_csv(import_csv)
# We Combine Imports and Exports to vertically
commercial_df = pd.concat([imports_df, exports_df])
# Drop Source Column ()
commercial_df.drop(labels="SOURCE_ID", axis=1, inplace=True)
# Split Attribute Description
attr_split_df = commercial_df.ATTRIBUTE_DESC.str.split(pat=", ", expand=True)
# Add Columns back to comercial df
commercial_df["DIRECTION"] = attr_split_df.loc[:,0]
commercial_df["MEASURE"] = attr_split_df.loc[:,1]
# Drop 'ATTRIBUTE_DESC' column
commercial_df.drop(labels="ATTRIBUTE_DESC", axis=1, inplace=True)
commercial_df.HS_CODE.fillna(value=404, inplace=True)
commercial_df.head()

,HS_CODE,COMMODITY_DESC,GEOGRAPHY_CODE,GEOGRAPHY_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,1,46682,US Import,QTY
1,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,2,37354,US Import,QTY
2,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,3,26080,US Import,QTY
3,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,4,38737,US Import,QTY
4,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,5,25799,US Import,QTY


## Clean Temperature Data  
First, Load Data, then begin cleaning  
  
**CLEANING** 
1. Split 'dt' into columns 0 (year) , 1 (month), 2(day of month)
2. Merge temp_df and new date_columns on index ( same size --> same index )
3. Rename numeric columns to 'YEAR_ID', 'MONTH_ID', 'DAY_ID' (respectively)
4. Drop 'dt' column (it is redundant)
5. Group by Country/Year/Month (to match commercial data) This means we are combining temperatures from different cities, which is a conceptual concession we had to make to match our other dataset
6. Aggregate (average), only keep temperature. We are dropping uncertainty here because we don't know what to do with it and how aggregation affects its statistical significance. 
7. Reset Indices 
  
The resulting DataFrame is kept separately as 'agg_temp_df' and will be used for the remaining steps

In [3]:
# LOAD DATA
temp_df = pd.read_csv(temp_csv)
# Split 'dt' column into Year, Month, Day
temp_dates = temp_df.dt.str.split(pat="-", expand=True)
# Merge on index
temp_df = pd.merge(left=temp_df, right=temp_dates, how="left", left_index=True, right_index=True)
# Rename columns
temp_df.rename(columns={0:"YEAR_ID", 1:"MONTH_ID", 2:"DAY_ID"}, inplace=True)
# Drop 'dt'
temp_df.drop(labels="dt", axis=1, inplace=True)
# Group by country/year/month
temp_gb = temp_df.groupby(by=["Country", "YEAR_ID", "MONTH_ID"])
# Aggregate by average, only keep temperature
agg_temp_df = temp_gb["AverageTemperature"].mean()
# Reset Indices
agg_temp_df = agg_temp_df.reset_index()
agg_temp_df.head()

,Country,YEAR_ID,MONTH_ID,AverageTemperature
0,Afghanistan,1833,01,2.290
1,Afghanistan,1833,02,3.319
2,Afghanistan,1833,03,7.444
3,Afghanistan,1833,04,13.576
4,Afghanistan,1833,05,19.321


## Align Country Names  
1. Make sets of country names from commercial_df and agg_temp_df. See how many are in each set. 
2. See how many countries are in the intersection
3. List the countries that are in agg_temp_df, but not in commercial_df. This is a good start on data cleaning.
4. Create dictionary maps to rename certain values in each dataframe. Then .replace 
5. Explore some of the interesting country labels
6. GENERAL CLEANING: Replace relevant Geography codes, many of which were identified at the loading stage

In [4]:
# Let's make sets of the country names
countries_com = set(commercial_df.GEOGRAPHY_DESC)
countries_temp = set(agg_temp_df.Country)
print(f"There are {len(countries_com)} country names in the commercial dataset")
print(f"There are {len(countries_temp)} country names in the temperature dataset")

There are 225 country names in the commercial dataset
There are 49 country names in the temperature dataset


In [5]:
#Lets see how many of the countries are in both datasets
countries_inter = countries_com.intersection(countries_temp)
print(f"There are {len(countries_inter)} country names that appear in both datasets")

There are 39 country names that appear in both datasets


In [6]:
# Since most of the temp countries fit inside of the commercial, let's see which ones do not
print("The following country names from the temperature data could not be found in the commercial data:")
print_statement = [print(each) for each in countries_temp.difference(countries_com)]

The following country names from the temperature data could not be found in the commercial data:
Taiwan
Congo (Democratic Republic Of The)
Germany
China
United States
South Africa
Côte D'Ivoire
Burma
Syria
Sudan


In [7]:
# We are going to rename some values of each df to be the cleanest possible
com_country_mapper = { 
    "Myanmar (Burma)" : "Burma",
    "China (Mainland)" : "China",
    "China (Taiwan)" : "Taiwan",
    "Congo (Kinshasa)" : "Congo (Democratic Republic Of The)",
    "Zaire" : "Congo (Democratic Republic Of The)",
    "German Democratic Republic" : "Germany",
    "Germany, Fed. Republic" : "Germany",
    "Republic of South Africa" : "South Africa",
}
agg_temp_country_mapper = {
    "Côte D'Ivoire" : "Ivory Coast", #This is the superior label
}

commercial_df.GEOGRAPHY_DESC.replace(com_country_mapper, inplace=True)
agg_temp_df.Country.replace(agg_temp_country_mapper, inplace=True)

Interestingly, there were no aquacultural exports to Sudan or Syria during this time

In [8]:
# And now the only countries that remain unmatched...
countries_com = set(commercial_df.GEOGRAPHY_DESC)
countries_temp = set(agg_temp_df.Country)
print("These Countries could not be matched:")
print_statement = [print(each) for each in countries_temp.difference(countries_com)]

These Countries could not be matched:
Sudan
Syria
United States


There is just one issue remaining. We mapped 2 countries (The Germanies) onto one country name. We should also map their 'GEOGRAPHY_CODE' onto one ID as well. Let's see what the IDs are for Germany:

*EDIT: We Also found a duplicate Serbia, which must be consolidated and a duplicates for Congo/Zaire were found manually*

In [9]:
commercial_df[commercial_df.GEOGRAPHY_DESC == "Germany"]["GEOGRAPHY_CODE"].value_counts()

4280    9574
4290       8
Name: GEOGRAPHY_CODE, dtype: int64

It looks like there are only a few trades with '4290' code. This makes sense because the 'German Democratic Republic' was dissolved about a year into our commercial dataset. Let's check our intuitions...

In [10]:
commercial_df[commercial_df.GEOGRAPHY_CODE == 4290]

,HS_CODE,COMMODITY_DESC,GEOGRAPHY_CODE,GEOGRAPHY_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE
319090,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,KG,1989,11,16000,US Import,QTY
319091,306130020.0,"SHRIMPS AND PRAWNS, SHELL-ON, FROZEN",4290,Germany,U.S.$,1989,11,122171,US Import,VLU
319092,404.0,"Shrimp, frozen",4290,Germany,KG,1989,11,16000,US Import,QTY
319093,404.0,"Shrimp, frozen",4290,Germany,U.S.$,1989,11,122171,US Import,VLU
319094,404.0,"Shrimp, total",4290,Germany,KG,1989,11,16000,US Import,QTY
319095,404.0,"Shrimp, total",4290,Germany,U.S.$,1989,11,122171,US Import,VLU
240232,301100000.0,"FISH, ORNAMENTAL, LIVE",4290,Germany,U.S.$,1989,1,2501,US Export,VLU
240233,404.0,Ornamental fish,4290,Germany,U.S.$,1989,1,2501,US Export,VLU


And there we go! Isn't history fun? Ok let's get back to the Data Cleaning. We are just going to map those few communist Germans into their new united Germany. (Also the Congos and Serbias)

In [11]:
# Replace Trouble codes
commercial_df.GEOGRAPHY_CODE.replace({4290 : 4280} , inplace=True) #Germanies
commercial_df.GEOGRAPHY_CODE.replace({4802 : 4801} , inplace=True) #Serbias

## Add Fish Type and Unique Commodity Identifier
We had to do some manual categorization to apply a 'fish type'. We are interested in categories here because many of the products are very similar.  
Additionally, we needed to add commodity_id to our schema because many commodity descriptions or hs codes were null duplicated

### Fish Type

In [12]:
# Define separate df
fish_type_df = commercial_df.copy()

# Pull COMMODITY_DESC into list
unique_commodity_list = fish_type_df.COMMODITY_DESC.unique().tolist()

# Define fish types
fish_list = ["Salmon", "Clam", "Shrimp", "Mussel", "Oyster", "Trout", "Scallop", "Ornamental", "Tilapia", "Carp"]

# Create empty dictionary
commodity_dict = {}

# Iterate through unique list and assign fish value
for commodity in unique_commodity_list:
    x=0
    while x < len(fish_list):
        if fish_list[x] in commodity:
            commodity_dict[commodity] = fish_list[x]
            x = x + len(fish_list)
        elif fish_list[x].upper() in commodity:
            commodity_dict[commodity] = fish_list[x]
            x = x + len(fish_list)
        elif fish_list[x].lower() in commodity:        
            commodity_dict[commodity] = fish_list[x]
            x = x + len(fish_list)
        else:
            x=x+1 
            
# Pull COMMODITY_DESC from commercial_df. Turn into list
all_commodity = fish_type_df.COMMODITY_DESC.tolist()
fish = []

# This is a list of descriptions with issues
problem_desc = ['PINK (HUMPIE)SALMN WHOLE/PIECES IN OIL/AIRTIT CNTR']

# Iterate through commodities and assign fish value
for commodity in all_commodity:
    for problem in problem_desc:
        if commodity == problem:
            fish.append("Salmon")
        else:
            append = commodity_dict[commodity]
            fish.append(append)

# Insert new column
fish_type_df.insert(2, "FISH_TYPE", fish)
fish_type_df.head()

,HS_CODE,COMMODITY_DESC,FISH_TYPE,GEOGRAPHY_CODE,GEOGRAPHY_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",Trout,1,World,KG,1989,1,46682,US Import,QTY
1,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",Trout,1,World,KG,1989,2,37354,US Import,QTY
2,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",Trout,1,World,KG,1989,3,26080,US Import,QTY
3,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",Trout,1,World,KG,1989,4,38737,US Import,QTY
4,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",Trout,1,World,KG,1989,5,25799,US Import,QTY


### Commodity Identifier

In [13]:
# Groupby combination of fields (we are going to drop aggregation)
unique_commodity_df = fish_type_df.groupby(by=["HS_CODE", "COMMODITY_DESC","FISH_TYPE"]).agg({'AMOUNT': 'sum'})
# Drop Aggregation
unique_commodity_df = unique_commodity_df.drop(columns=["AMOUNT"])
# Reset Index Twice
unique_commodity_df.reset_index(inplace=True) #First to get indexes back to fields
unique_commodity_df.reset_index(inplace=True) #Second to extract index
# Rename Index to commodity_id
unique_commodity_df.rename({"index":"COMMODITY_ID"},axis="columns", inplace=True)
unique_commodity_df.head()

,COMMODITY_ID,HS_CODE,COMMODITY_DESC,FISH_TYPE
0,0,404.0,"Atlantic salmon, fillets",Salmon
1,1,404.0,"Atlantic salmon, fresh",Salmon
2,2,404.0,"Atlantic salmon, frozen",Salmon
3,3,404.0,"Atlantic salmon, total",Salmon
4,4,404.0,Clams,Clam


In [14]:
# Now we want to merge this into a new, fully expanded DataFrame using our HS_CODE amd COMMODITY_DESC as indexes
super_commercial_df = pd.merge(left=commercial_df, right=unique_commodity_df, how="left", on=["HS_CODE", "COMMODITY_DESC"])
super_commercial_df.head()

,HS_CODE,COMMODITY_DESC,GEOGRAPHY_CODE,GEOGRAPHY_DESC,UNIT_DESC,YEAR_ID,TIMEPERIOD_ID,AMOUNT,DIRECTION,MEASURE,COMMODITY_ID,FISH_TYPE
0,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,1,46682,US Import,QTY,33,Trout
1,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,2,37354,US Import,QTY,33,Trout
2,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,3,26080,US Import,QTY,33,Trout
3,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,4,38737,US Import,QTY,33,Trout
4,302110000.0,"TROUT (SALMO TRUTTA, S. CLARKI ETC) FRESH, CHI...",1,World,KG,1989,5,25799,US Import,QTY,33,Trout


## Table Creation  
We now prepare final tables for export. The cleaning process varies by table, as each table comes from a slightly unique source.  
This cleaning stage was made much easier (and more readable) because we determine the columns we would like with lists stored externally in the **rename_fields.py** file.

### Country Table
**IMPORTANT NOTE:** This table contains the country_code and country_name for all the countries **in the commerce dataset**. It does not contain comprehensive values for the temperature data. This means that SQL queries going *from* the temp table *to* the commerce table can have null values. **A preferred join would always be an INNER JOIN**

In [15]:
# The columns we want are keys of the dict
country_cols = list(country_renamer.keys())
# Group the Super Commercial Dataset by desired columns (we will drop aggregation)
country_df = super_commercial_df.groupby(by=country_cols).agg({'AMOUNT': 'sum'})
# Drop Aggregation
country_df = country_df.drop(columns=["AMOUNT"])
# Return indices to Fields
country_df.reset_index(inplace=True)
# Rename Fields in Final DataFrame
country_df_final = country_df.rename(mapper=country_renamer, axis=1)
country_df_final.head()

,country_code,country_name
0,1,World
1,1010,Greenland
2,1220,Canada
3,1610,St Pierre and Miquelon
4,2010,Mexico


### Fish Table 
This Table contains all of the descriptive information about the commodity in question. Honestly, it could easily have been included in the commercial table, but we felt it was worth showing a *somewhat* normal form for the Database

In [16]:
# The columns we want are keys of the dict
fish_cols = list(fish_renamer.keys())
# Group the Super Commercial Dataset by desired columns (we will drop aggregation)
fish_df_final = super_commercial_df.groupby(by=fish_cols).agg({'AMOUNT': 'sum'})
# Drop Aggregation
fish_df_final = fish_df_final.drop(columns=["AMOUNT"])
# Return indices to Fields
fish_df_final = fish_df_final.reset_index()
# Rename Fields in Final DataFrame
fish_df_final = fish_df_final.rename(mapper=fish_renamer, axis=1)
fish_df_final.commodity_id = fish_df_final.commodity_id.astype(int)
fish_df_final.head()

,commodity_id,hs_code,commodity_desc,fish_type
0,0,404.0,"Atlantic salmon, fillets",Salmon
1,1,404.0,"Atlantic salmon, fresh",Salmon
2,2,404.0,"Atlantic salmon, frozen",Salmon
3,3,404.0,"Atlantic salmon, total",Salmon
4,4,404.0,Clams,Clam


### Commercial Table
This table contains only the transactional data. Again, data has been separated for normalization purposes.

In [17]:
# The columns we want are keys of the dict
commercial_cols = list(commercial_renamer.keys())
# No need to group the data here
commercial_df_final = super_commercial_df[commercial_cols]
# Rename Fields in Final DataFrame
commercial_df_final = commercial_df_final.rename(mapper=commercial_renamer, axis=1)
commercial_df_final.commodity_id = commercial_df_final.commodity_id.astype(int)
commercial_df_final.head()

,country_code,commodity_id,unit_descr,direction,measure,year_id,month_id,amount
0,1,33,KG,US Import,QTY,1989,1,46682
1,1,33,KG,US Import,QTY,1989,2,37354
2,1,33,KG,US Import,QTY,1989,3,26080
3,1,33,KG,US Import,QTY,1989,4,38737
4,1,33,KG,US Import,QTY,1989,5,25799


### Temp Table 
This table contains the aggregated temperatures for all of the countries with data, across all time, regardless of whether those countries or dates exist in the other dataset

In [18]:
# The columns we want are keys of the dict
temp_cols = list(temp_renamer.keys())
# No need to group the data here
temp_df_final = agg_temp_df[temp_cols]
# Rename Fields in Final DataFrame
temp_df_final = agg_temp_df.rename(mapper=temp_renamer, axis=1)
temp_df_final.head()

,country_name,year_id,month_id,avg_temperature
0,Afghanistan,1833,01,2.290
1,Afghanistan,1833,02,3.319
2,Afghanistan,1833,03,7.444
3,Afghanistan,1833,04,13.576
4,Afghanistan,1833,05,19.321


In [19]:
# EXPORT TABLES TO CSV
country_df_final.to_csv(TABLE_country, index=False)
fish_df_final.to_csv(TABLE_fish, index=False)
commercial_df_final.to_csv(TABLE_commercial, index=False)
temp_df_final.to_csv(TABLE_temp, index=False)

## Aggregation 
Aggregations that will not be loaded into the database but are here for academic purposes

In [25]:
#Grouping by Country, Import/Export and QTY/VLU
country_preference = super_commercial_df.groupby(['GEOGRAPHY_DESC', "FISH_TYPE","DIRECTION", "UNIT_DESC"]).agg({'AMOUNT': 'sum'})
country_preference.sort_

AMOUNT
GEOGRAPHY_DESC FISH_TYPE  DIRECTION UNIT_DESC         
Afghanistan    Ornamental US Export U.S.$        17370
                          US Import U.S.$        15548
               Oyster     US Export KG           66010
                                    U.S.$       728718
               Salmon     US Export KG           27366
...                                                ...
Zambia         Ornamental US Import U.S.$      5221980
Zimbabwe       Salmon     US Export KG             868
                                    U.S.$         5708
               Tilapia    US Import KG          350247
                                    U.S.$      1187976

[2951 rows x 1 columns]

In [30]:
species_df = super_commercial_df.groupby(by=["FISH_TYPE", "DIRECTION", "UNIT_DESC"]).agg({'AMOUNT': 'sum'})
species_df.sort_values(by="AMOUNT", ascending=False)

AMOUNT
FISH_TYPE  DIRECTION UNIT_DESC              
Shrimp     US Import U.S.$      621051473184
Salmon     US Import U.S.$      177139869316
           US Export U.S.$       79354118492
Shrimp     US Import KG          73718844020
Tilapia    US Import U.S.$       61335791916
Salmon     US Import KG          26777225914
Scallop    US Import U.S.$       24049587952
Salmon     US Export KG          19386450324
Tilapia    US Import KG          16479140478
Shrimp     US Export U.S.$       14486065852
Scallop    US Export U.S.$        8807626444
Mussel     US Import U.S.$        5698530844
Oyster     US Import U.S.$        5235793088
Clam       US Import U.S.$        5154108476
Ornamental US Import U.S.$        4443298826
Clam       US Export U.S.$        3662031580
Trout      US Import U.S.$        2979136936
Scallop    US Import KG           2511221644
Mussel     US Import KG           2008522272
Shrimp     US Export KG           1690562650
Clam       US Import KG           1540970468
Oyster     US Export U.S.$        1445085016
Ornamental US Export U.S.$        1179998528
Oyster     US Import KG            996582304
Scallop    US Export KG            694566332
Trout      US Import KG            481940388
Clam       US Export KG            459988088
Trout      US Export U.S.$         451359660
Oyster     US Export KG            239604244
Mussel     US Export U.S.$         220520608
Carp       US Import U.S.$         142815510
Mussel     US Export KG             72270536
Trout      US Export KG             72220332
Oyster     US Export L               5901860
           US Import L                228076